In [33]:
import numpy as np
from scipy.optimize import linear_sum_assignment

In [55]:
def stabilize(confidence, very_small_value=1e-8, dtype=np.float32):
    confidence[confidence > 1.0]  = 1.0
    confidence[confidence <= 0.0]  = very_small_value
    return confidence.astype(dtype)

In [104]:

def loss(predictions, truths, alpha=1.0):
    """
    predictions is a float matrix with the dimensions [batch_size, num_predictions, 5]
        - the '5' represents x, y, height, width & confidence of each prediction of each batch item
        
    truths is a float matrix with the dimensions [batch_size, num_ground_truths, 4]
        - the '4' represents x, y, height & width of each ground truth of each batch item
    
    for all items in the batch, num_predictions will be the same 
        (as determined by the image sizes in the batch and the number of bounding 
        boxes we want to predict per location)
        
    for all items in the batch, num_ground_truths will be different for different images
    
    """
    
    assert len(predictions) == len(truths), 'Batch size of predictions and ground truths do not match'
        
    predictions = np.array(predictions)
    truths      = np.array(truths)
    batch_size  = len(predictions)
    the_loss    = None
    
    for batch_idx in range(batch_size):
        
        # Create the cost matrix: predictions vs ground truths
        ######################################################
        
        num_preds = len(predictions[batch_idx])
        num_truths = len(truths[batch_idx])
        
        # For numerical stability
        predictions[batch_idx][:, 4] = stabilize(predictions[batch_idx][:, 4])
        
        # extract (1-confidence) values
        one_minus_confidence = np.expand_dims(1.0 - predictions[batch_idx][:, 4], axis=1)
        
        # For numerical stability
        one_minus_confidence = stabilize(one_minus_confidence)
        
        # the cost matrix, initialized to 0
        cost_matrix = np.zeros((num_preds, num_truths))
        
        # fill the cost matrix
        # !! TODO - Vectorize this loop !!
        for p in range(num_preds):
            for t in range(num_truths):
                localization_cost = alpha * np.linalg.norm(predictions[batch_idx][p][:4] - truths[batch_idx][t])**2
                confidence_cost = -1 * np.log(predictions[batch_idx][p][4])
                cost_matrix[p, t] = localization_cost + confidence_cost
        
        # run the hungarian algorithm
        row_ind, col_ind = linear_sum_assignment(cost_matrix) 
        
        # print(np.round(cost_matrix, 2), '\n')
        # Calculate the loss
        ######################################################
        # !! TODO - Vectorize this loop !!
        for t in range(num_truths):
            loop_loss = 0.0
            for p in range(num_preds):
                if t in col_ind and p in row_ind:
                    loop_loss += cost_matrix[p][t]
                    #print('1', cost_matrix[p][t])
                else:
                    loop_loss -= np.log(one_minus_confidence[p])
                    #print('2', np.log(one_minus_confidence[p]))
            
            #print(loop_loss)
            
            if the_loss is None:
                the_loss = 0.0
            
            the_loss += loop_loss
        
    
    return the_loss

In [108]:
p = np.array([[[10, 20, 30, 40, .001], [1, 2, 3, 4, .99]]])
g = np.array([[[1, 2, 3, 4]]])
loss(p, g)

array([0.01105081], dtype=float32)

In [102]:
0.01 + (-np.log(1-0.9))

2.3125850929940457

In [46]:
p = np.array([[1, 1.1], [3, 0.6], [5, -1.0]])
p[:, 1] = stability(p[:, 1])
col_to_add = np.expand_dims(1.0 - p[:, 1], axis=1)
print(col_to_add)
val = np.append(p, col_to_add, axis=1)
np.round(val, 2)

[[0.        ]
 [0.39999998]
 [0.99999999]]


array([[1. , 1. , 0. ],
       [3. , 0.6, 0.4],
       [5. , 0. , 1. ]])

In [5]:
cost = None
if cost is None:
    cost = 0
cost += 1
print(cost)

1
